<h1>Machine Learning approach to achieve efficient DMFT solution<h1>

<p>We use DMFT with exact diagonalization (ED) as impurity solver<p> 
<p>ED solver require intinsic parameter called bath orbital obtain using optization fit to bath funtion<p>

$$\mathcal{G}^0(i\omega_n) = -i\omega_n+T +\sum_i\frac{V_i}{i\omega_n+\epsilon_i}$$ 
    
    
    
<p>where 'i' is the number of bath orbital<p>

In [ ]:
#----Initialization----------------------
import numpy as np
import pandas as pd
import tensorflow
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from scipy.special import softmax
from scipy import sparse
import math
pi=math.pi
exp=math.exp

<p> Idea is to get the green's funtion for low number of orbital and train a model with an output with 8 obital<p>
 <p> $$i\omega_n=256$$, $$U=3-5 (100)$$ and $$N_b=2,3,6$$<p>

In [ ]:
inp_up=np.loadtxt('Gnfnup.dat') # input with 2,4,6 orbital 100 random U
inp_dw=np.loadtxt('Gnfndw.dat')
out_up=np.loadtxt('Gfup.dat') # output with 8 orbital 100
out_dw=np.loadtxt('Gfdw.dat')
#------------------To cross check we reshape-------------
Ginp_up=np.reshape(inp_up, (100,256*3), order='C')
Ginp_dw=np.reshape(inp_dw, (100,256*3),order='C')
Gout_up=np.reshape(out_up, (100,256),order='C')
Gout_dw=np.reshape(out_dw, (100,256),order='C')
plt.plot(Ginp_up[0,:],'o-')
plt.show()
tmp=np.reshape(inp_up, (100,256,3), order='C')
tmpdw=np.reshape(inp_dw, (100,256,3), order='C')
Ginp_uporb=pd.DataFrame(Ginp_up)   
display(Ginp_uporb)

In [ ]:

Ginp_uporb.hist(column=2, sharey=True, figsize=(8,8))
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 2, sharex='col', sharey='row',figsize=(8, 8))
ax[0,0].plot(tmp[5,:,0],'-',label=r'$\uparrow$;2')
ax[0,0].plot(tmpdw[5,:,0],'-',label=r'$\downarrow$;2')
ax[0,0].plot(tmp[5,:,1],'-',label=r'$\uparrow$;4')
ax[0,0].plot(tmpdw[5,:,1],'-',label=r'$\downarrow$;4')
ax[0,0].plot(tmp[5,:,2],'-',label=r'$\uparrow$;6')
ax[0,0].plot(tmpdw[5,:,2],'-',label=r'$\downarrow$;6')
ax[0,0].set(xlabel=r'$\tau$')
ax[0,0].legend(loc='center')
ax[0,0].set(ylabel=r'$G(\tau)$')
ax[0, 0].set_title("U=U1")

ax[0,1].plot(tmp[20,:,0],'-',label=r'$\uparrow$;2')
ax[0,1].plot(tmpdw[20,:,0],'-',label=r'$\downarrow$;2')
ax[0,1].plot(tmp[20,:,1],'-',label=r'$\uparrow$;4')
ax[0,1].plot(tmpdw[20,:,1],'-',label=r'$\downarrow$;4')
ax[0,1].plot(tmp[20,:,2],'-',label=r'$\uparrow$;6')
ax[0,1].plot(tmpdw[20,:,2],'-',label=r'$\downarrow$;6')
ax[0,1].set(xlabel=r'$\tau$')
ax[0,1].legend(loc='center')
ax[0,1].set(ylabel=r'$G(\tau)$')
ax[0,1].set_title("U=U2")
ax[1,0].plot(tmp[60,:,0],'-',label=r'$\uparrow$;2')
ax[1,0].plot(tmpdw[60,:,0],'-',label=r'$\downarrow$;2')
ax[1,0].plot(tmp[60,:,1],'-',label=r'$\uparrow$;4')
ax[1,0].plot(tmpdw[60,:,1],'-',label=r'$\downarrow$;4')
ax[1,0].plot(tmp[60,:,2],'-',label=r'$\uparrow$;6')
ax[1,0].plot(tmpdw[60,:,2],'-',label=r'$\downarrow$;6')
ax[1,0].set(xlabel=r'$\tau$')
ax[1,0].legend(loc='center')
ax[1,0].set(ylabel=r'$G(\tau)$')
ax[1,0].set_title("U=U3")
ax[1,1].plot(tmp[80,:,0],'-',label=r'$\uparrow$;2')
ax[1,1].plot(tmpdw[80,:,0],'-',label=r'$\downarrow$;2')
ax[1,1].plot(tmp[80,:,1],'-',label=r'$\uparrow$;4')
ax[1,1].plot(tmpdw[80,:,1],'-',label=r'$\downarrow$;4')
ax[1,1].plot(tmp[80,:,2],'-',label=r'$\uparrow$;6')
ax[1,1].plot(tmpdw[80,:,2],'-',label=r'$\downarrow$;6')
ax[1,1].set(xlabel=r'$\tau$')
ax[1,1].legend(loc='center')
ax[1,1].set(ylabel=r'$G(\tau)$')
ax[1,1].set_title("U=U4")
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 2, sharex='col', sharey='row',figsize=(8, 8))
ax[0,0].plot(Gout_up[2,:],'-',label=r'$\uparrow$')
ax[0,0].plot(Gout_dw[2,:],'-',label=r'$\downarrow$')
ax[0,0].set(xlabel=r'$\tau$')
ax[0,0].legend(loc='center')
ax[0,0].set(ylabel=r'$G(\tau)$')
ax[0,1].plot(Gout_up[10,:],'-',label=r'$\uparrow$')
ax[0,1].plot(Gout_dw[10,:],'-',label=r'$\downarrow$')
ax[0,1].set(xlabel=r'$\tau$')
ax[0,1].legend(loc='center')
ax[0,1].set(ylabel=r'$G(\tau)$')
ax[1,0].plot(Gout_up[20,:],'-',label=r'$\uparrow$')
ax[1,0].plot(Gout_dw[20,:],'-',label=r'$\downarrow$')
ax[1,0].set(xlabel=r'$\tau$')
ax[1,0].legend(loc='center')
ax[1,0].set(ylabel=r'$G(\tau)$')

ax[1,1].plot(Gout_up[40,:],'-',label=r'$\uparrow$')
ax[1,1].plot(Gout_dw[40,:],'-',label=r'$\downarrow$')
ax[1,1].set(xlabel=r'$\tau$')
ax[1,1].legend(loc='center')
ax[1,1].set(ylabel=r'$G(\tau)$')
plt.show()

In [ ]:
def randomgrin(n):
    import random
    sin=np.zeros((100,256*3))
    np.random.seed(n)
    for i in range (100):
        sin[i,:]=np.random.normal(0.5,0.1)
    sin=sin/(np.amax(sin)*100)
    return sin

In [ ]:
Ginp_up_s1=Ginp_up+randomgrin(1)
Ginp_up_s2=Ginp_up+randomgrin(2)
Ginp_up_s3=Ginp_up+randomgrin(3)
Ginp_up_s4=Ginp_up+randomgrin(4)
Ginp_up_s5=Ginp_up+randomgrin(5)

In [ ]:
def randomgrout(n):
    import random
    sout=np.zeros((100,256))
    np.random.seed(n)
    for i in range (100):
        sout[i,:]=np.random.normal(0.5,0.1)
    sout=sout/(np.amax(sout)*100)
    return sout

In [ ]:
Gout_up_s1=Gout_up+randomgrout(1)
Gout_up_s2=Gout_up+randomgrout(2)
Gout_up_s3=Gout_up+randomgrout(3)
Gout_up_s4=Gout_up+randomgrout(4)
Gout_up_s5=Gout_up+randomgrout(5)

In [ ]:
X=np.vstack((Ginp_up,Ginp_up_s1,Ginp_up_s2,Ginp_up_s3,Ginp_up_s4,Ginp_up_s5))
y=np.vstack((Gout_up,Gout_up_s1,Gout_up_s2,Gout_up_s3,Gout_up_s4,Gout_up_s5))

In [ ]:
df = pd.DataFrame(data=X)

In [ ]:
df2=df.round(2)
X=df2.to_numpy()

In [ ]:
#-------------------------

#print (X)
print(X.shape, y.shape)
X_train=X[0:500,:]
X_test=X[500:600,:]
y_train=y[0:500,:]
y_test=y[500:600,:]
print(y_train.shape)


In [ ]:
import pandas as pd
df=pd.DataFrame(X)
#display(df)
#df.hist(column=0,bins=50, grid=False, figsize=(10,6), color='#86bf91', zorder=2, rwidth=0.9)
ax = df.hist(column=0, bins=100, grid=False, figsize=(4,4), color='#86bf91', zorder=2, rwidth=0.9)
ax = ax[0]
for x in ax:

    # Despine
    x.spines['right'].set_visible(True)
    x.spines['top'].set_visible(True)
    x.spines['left'].set_visible(True)

    # Switch off ticks
    x.tick_params(axis="both", which="both", bottom="on", top="on", labelbottom="on", left="on", right="on", labelleft="on")
    # Draw horizontal axis lines
    vals = x.get_yticks()
    for tick in vals:
        x.axhline(y=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)

    # Remove title
    x.set_title("")

    # Set x-axis label
    x.set_xlabel(r"$G(\tau)$", labelpad=20, weight='bold', size=12)

    # Set y-axis label
    x.set_ylabel("units", labelpad=20, weight='bold', size=12)

    

In [ ]:
columns=[]
for i in range(768):
    x="col"+"_"+str(i)
    columns.append(x)
columns

In [ ]:
df.columns=columns
df.index
#display(df)
#df.reset_index().plot(x='index', y='col_4',figsize=(8,8))
#plt.show()



<h2>For now, we do not use PCA<h2> 

In [ ]:
from sklearn.decomposition import PCA, KernelPCA
scikit_pca = PCA(n_components=3)
scikit_pca.fit(X_train)
X_pca_train=scikit_pca.transform(X_train)
print(X_pca_train.shape)
X_train_w=scikit_pca.components_
print(scikit_pca.components_)

scikit_pca.fit(X_test)
X_pca_test=scikit_pca.transform(X_test)
print(X_pca_test.shape)
print(scikit_pca.components_)
print(scikit_pca.components_.shape)
X_test_w=scikit_pca.components_
plt.plot(X_pca_test)

<p> Tensor network Keras to define the netwoork <p> 

In [ ]:
from keras.layers import GaussianNoise
from tensorflow.keras.regularizers import l2
# define the Neural Network
model = Sequential()
model.add(Dense(32, activation = 'relu',input_shape=(256*3,))) 
model.add(Dropout(0.2))
model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(256))

In [ ]:
tensorflow.keras.utils.plot_model(model)


In [ ]:
print(model.summary())

In [ ]:
# Lets look at the model's prediction before training it
pred = model(X_train)
print(pred.shape)
#plt.plot(pred)

In [ ]:
# here you compile the model and choose the optimizer and loss for the training
#opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(
   loss = 'mse', 
   optimizer = RMSprop(), 
   metrics = ['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

history = model.fit(
   X_train, y_train,    
   batch_size=10, 
   epochs = 1000, 
   verbose = 1, 
   validation_data=(X_test,y_test), 
   #callbacks = [EarlyStopping(monitor = 'val_loss', patience = 20)]
)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
prediction = model.predict(X_test) 
print(prediction.shape) 
ypred=prediction
print(ypred.shape)
#for i in range(0,80):
#    plt.plot(ypred[i,:],'-')
    
#   print(ypred[i,-1],ypred[i,0])
i=40

#plt.plot(prediction.flatten())
plt.plot(y_test[i,:],'o')
plt.plot(ypred[i,:],'-')
plt.legend(['test', 'pred'], loc='best')

#print(ypred[i,-1],ypred[i,0])
#print(y_test[i,-1],y_test[i,0])
#plt.plot(ypred.flatten(),'o')
#plt.plot(y_test.flatten(),'*')
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='valLoss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn import metrics
from math import sqrt
meanAbErr = metrics.mean_absolute_error(y_test, ypred)
meanSqErr = metrics.mean_squared_error(y_test, ypred)
rootMeanSqErr = np.sqrt(metrics.mean_squared_error(y_test, ypred))

print('Mean Absolute Error:', meanAbErr)
print('Mean Square Error:', meanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print("%s: %.4f%%" % (model.metrics_names[1], score[1]*100))